In [41]:
import pandas as pd
import time


# BEGIN PROGRAM
    
print("2020 ACT election simulator")

# read in parameters
print("\nreading in parameters ...")
groups = pd.read_csv("./data/Groups.txt")
candidates = pd.read_csv("./data/Candidates.txt")
electorates = pd.read_csv("./data/Electorates.txt")
print("complete\n")

# read in votes
ballots = pd.DataFrame(columns=["pindex", "pref", "pcode", "ccode", "ecode"])
for electorate in electorates.index:
    print(f"reading in {electorates.loc[electorate].electorate} ballot papers ...")
    add_ballots = pd.read_csv("./data/YerrabiTotal.txt", usecols=["pindex", "pref", "pcode", "ccode"])
    add_ballots["ecode"] = electorates.loc[electorate].electorate
    ballots = ballots.append(add_ballots)
print("complete")

ballots

# create temporary vote sample
# yerrabi = yerrabi.sample(frac=.1, random_state=1)

# print("\nfiltering data ...")

# # limit datasets to Yerrabi-only
# groups = groups[groups["ecode"] == 5].set_index("pcode")
# candidates = candidates[candidates["ecode"] == 5]
# print("complete")

# print("\ncreating candidate IDs ...")

# # create candidate IDs
# candidates["votes"] = [0] * len(candidates)
# candidates["primary"] = [0] * len(candidates)
# candidates["cname"] = candidates["cname"].apply(lambda x: x.split(", ")[1] + " " + x.split(", ")[0])
# candidates["party"] = candidates["pcode"].apply(lambda x: groups.loc[x].pabbrev)
# candidates = create_ids(candidates)
# candidates.set_index("id", inplace=True)
# candidates = candidates[["cname", "party", "primary", "votes"]]

# print("\npreparing preference data ...")

# # assign candidate IDs to each vote preference
# yerrabi = create_ids(yerrabi)
# yerrabi[["pindex", "pref", "pcode", "id"]]

# print("\ncreating voter files ...")

# # group vote preferences into individual voters
# voters = pd.DataFrame(index=yerrabi.pindex.unique(), columns=["votes", "pref", "value"])
# start_time = time.time()
# time_interval = 10
# for i, indice in enumerate(voters.index):
#     data = yerrabi[yerrabi["pindex"] == indice].sort_values("pref")
#     votes = []
#     for j in data.index:
#         votes.append(data.loc[j].id)
#     voters.loc[indice].votes = votes
#     voters.loc[indice].pref = 0
#     voters.loc[indice].value = 1
#     if (time.time() - start_time) > time_interval:
#         print(f"{(i + 1) / len(voters.index):.1%} processed")
#         time_interval = time_interval + 10
# print("complete")

# quota = len(voters) / 6 + 1
# print(f"\nquota established: {quota:.1f}")

# print("\ncreate final lists")

# elected_candidates = pd.DataFrame(columns=["cname", "party", "primary"])
# eliminated_candidates = pd.DataFrame(columns=["cname", "party", "primary"])
# exhausted_votes = 0
# print("complete")

# print("\ndistribute initial votes ...")

# # begin counting rounds
# count_no = 1
# assign_votes()

2020 ACT election simulator

reading in parameters ...
complete

reading in Brindabella ballot papers ...
reading in Ginninderra ballot papers ...
reading in Kurrajong ballot papers ...
reading in Murrumbidgee ballot papers ...
reading in Yerrabi ballot papers ...
complete


,pindex,pref,pcode,ccode,ecode
0,71141745,1,2,2,Brindabella
1,71141745,2,3,1,Brindabella
2,71141745,3,3,2,Brindabella
3,71141745,4,2,4,Brindabella
4,71141745,5,7,1,Brindabella
...,...,...,...,...,...
355370,1123849065,3,4,1,Yerrabi
355371,1123849065,4,5,1,Yerrabi
355372,1123849065,5,2,1,Yerrabi
355373,1814474115,1,1,2,Yerrabi
